In [1]:
import numpy as np
import sympy as sp
from scipy import interpolate as interp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
from scipy import interpolate
import PIL
from scipy import misc
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200


In [2]:
def bicubic_interpolation2(xi, yi, zi, xnew, ynew):

    # check sorting
    if np.any(np.diff(xi) < 0) and np.any(np.diff(yi) < 0) and\
    np.any(np.diff(xnew) < 0) and np.any(np.diff(ynew) < 0):
        raise ValueError('data are not sorted')

    if zi.shape != (xi.size, yi.size):
        raise ValueError('zi is not set properly use np.meshgrid(xi, yi)')

    z = np.zeros((xnew.size, ynew.size))

    deltax = xi[1] - xi[0]
    deltay = yi[1] - yi[0]
    for n, x in enumerate(xnew):
        for m, y in enumerate(ynew):

            if xi.min() <= x <= xi.max() and yi.min() <= y <= yi.max():

                i = np.searchsorted(xi, x) - 1
                j = np.searchsorted(yi, y) - 1

                x1  = xi[i]
                x2  = xi[i+1]

                y1  = yi[j]
                y2  = yi[j+1]

                px = (x-x1)/(x2-x1)
                py = (y-y1)/(y2-y1)

                f00 = zi[i-1, j-1]      #row0 col0 >> x0,y0
                f01 = zi[i-1, j]        #row0 col1 >> x1,y0
                f02 = zi[i-1, j+1]      #row0 col2 >> x2,y0

                f10 = zi[i, j-1]        #row1 col0 >> x0,y1
                f11 = p00 = zi[i, j]    #row1 col1 >> x1,y1
                f12 = p01 = zi[i, j+1]  #row1 col2 >> x2,y1

                f20 = zi[i+1,j-1]       #row2 col0 >> x0,y2
                f21 = p10 = zi[i+1,j]   #row2 col1 >> x1,y2
                f22 = p11 = zi[i+1,j+1] #row2 col2 >> x2,y2

                if 0 < i < xi.size-2 and 0 < j < yi.size-2:

                    f03 = zi[i-1, j+2]      #row0 col3 >> x3,y0

                    f13 = zi[i,j+2]         #row1 col3 >> x3,y1

                    f23 = zi[i+1,j+2]       #row2 col3 >> x3,y2

                    f30 = zi[i+2,j-1]       #row3 col0 >> x0,y3
                    f31 = zi[i+2,j]         #row3 col1 >> x1,y3
                    f32 = zi[i+2,j+1]       #row3 col2 >> x2,y3
                    f33 = zi[i+2,j+2]       #row3 col3 >> x3,y3

                elif i<=0:

                    f03 = f02               #row0 col3 >> x3,y0

                    f13 = f12               #row1 col3 >> x3,y1

                    f23 = f22               #row2 col3 >> x3,y2

                    f30 = zi[i+2,j-1]       #row3 col0 >> x0,y3
                    f31 = zi[i+2,j]         #row3 col1 >> x1,y3
                    f32 = zi[i+2,j+1]       #row3 col2 >> x2,y3
                    f33 = f32               #row3 col3 >> x3,y3

                elif j<=0:

                    f03 = zi[i-1, j+2]      #row0 col3 >> x3,y0

                    f13 = zi[i,j+2]         #row1 col3 >> x3,y1

                    f23 = zi[i+1,j+2]       #row2 col3 >> x3,y2

                    f30 = f20               #row3 col0 >> x0,y3
                    f31 = f21               #row3 col1 >> x1,y3
                    f32 = f22               #row3 col2 >> x2,y3
                    f33 = f23               #row3 col3 >> x3,y3


                elif i == xi.size-2 or j == yi.size-2:

                    f03 = f02               #row0 col3 >> x3,y0

                    f13 = f12               #row1 col3 >> x3,y1

                    f23 = f22               #row2 col3 >> x3,y2

                    f30 = f20               #row3 col0 >> x0,y3
                    f31 = f21               #row3 col1 >> x1,y3
                    f32 = f22               #row3 col2 >> x2,y3
                    f33 = f23               #row3 col3 >> x3,y3

                Z = np.array([f00, f01, f02, f03,
                             f10, f11, f12, f13,
                             f20, f21, f22, f23,
                             f30, f31, f32, f33]).reshape(4,4).transpose()

                X = np.tile(np.array([-1, 0, 1, 2]), (4,1))
                X[0,:] = X[0,:]**3
                X[1,:] = X[1,:]**2
                X[-1,:] = 1

                Cr = Z@np.linalg.inv(X)
                R = Cr@np.array([px**3, px**2, px, 1])

                Y = np.tile(np.array([-1, 0, 1, 2]), (4,1)).transpose()
                Y[:,0] = Y[:,0]**3
                Y[:,1] = Y[:,1]**2
                Y[:,-1] = 1

                Cc = np.linalg.inv(Y)@R

                z[n,m]=(Cc@np.array([py**3, py**2, py, 1]))


    return z


In [3]:
def view(x, y, z):
    """
    imput image data where x and y are the coordinates of the image and z is the image data

    :param x: shape (n,) or (n,1) from image dimension
    :param y: shape (,m) or (1,m) from image dimension
    :param z: grayscale image, shape (n,m)
    :return:
    """
    # if type(x) == np.ndarray:
    #     x = x.tolist()
    # if type(y) == np.ndarray:
    #     y = y.tolist()

    # x = np.linspace(-6, 6, 11)
    # y = np.linspace(-6, 6, 11)

    xx, yy = np.meshgrid(x, y)

    x_new = np.linspace(0, 99, 100)
    y_new = np.linspace(0, 99, 100)

    xx_new, yy_new = np.meshgrid(x_new, y_new)

    z_new = bicubic_interpolation2(x, y, z, x_new, y_new)

    z_true = interpolate.griddata((xx.ravel(), yy.ravel()), z.ravel(), (xx_new, yy_new), method='cubic')

    # f_scipy = interpolate.interp2d(x, y, z, kind='cubic')
    f_scipy = interpolate.RectBivariateSpline(x, y, z, kx=3, ky=3)

    F = interpolate.PPoly.from_spline(f_scipy)

    z_scipy = f_scipy(x_new, y_new)

    fig, ax = plt.subplots(2, 2, sharey=True, figsize=(16,12))

    img0 = ax[0, 0].scatter(xx, yy, c=z, s=100)
    ax[0, 0].set_title('original points')
    fig.colorbar(img0, ax=ax[0, 0], orientation='vertical', shrink=1, pad=0.01)

    img1 = ax[0, 1].imshow(z_new, vmin=z_new.min(), vmax=z_new.max(), origin='lower',
               extent=[x_new.min(), x_new.max(), y_new.max(), y_new.min()])
    ax[0, 1].set_title('bicubic our code')
    fig.colorbar(img1, ax=ax[0, 1], orientation='vertical', shrink=1, pad=0.01)


    img2 = ax[1, 0].imshow(z_scipy, vmin=z_scipy.min(), vmax=z_scipy.max(), origin='lower',
               extent=[x_new.min(), x_new.max(), y_new.max(), y_new.min()])
    ax[1, 0].set_title('RectBivariateSpline scipy')
    # ax.pcolormesh(xx_new, yy_new, z_scipy, vmin=-0.25, vmax=0.25, shading='gouraud')
    fig.colorbar(img2, ax=ax[1, 0], orientation='vertical', shrink=1, pad=0.01)


    img3 = ax[1, 1].imshow(z_true, vmin=z_true.min(), vmax=z_true.max(), origin='lower',
               extent=[x_new.min(), x_new.max(), y_new.max(), y_new.min()])
    ax[1, 1].set_title('true model')
    fig.colorbar(img3, ax=ax[1, 1], orientation='vertical', shrink=1, pad=0.01)

    plt.subplots_adjust(wspace=0.05, hspace=0.15)

    plt.show()
    print(F)
    return F



In [4]:
if __name__ == '__main__':
    image = PIL.Image.open('img.png')
    image = image.crop((400, 400, 600, 600))
    x_size, y_size = image.size  # Get dimensions
    image = np.array(image)  # "image" is now an array of RGB pixels
    # image = image / 255
    #z = image[:, :, 0]
    z = image[:, :, 0]
    x = np.linspace(0, x_size - 1, x_size)
    y = np.linspace(0, y_size - 1, y_size)

    tck_poly = view(x, y, z)
    print(tck_poly[2])

TypeError: cannot unpack non-iterable RectBivariateSpline object

In [ ]:
vetor = pd.DataFrame(tck_poly[1])
vetor

In [ ]:
x, y = sp.symbols('x y')
a00, a01, a02, a03, a10, a11, a12, a13, a20, a21, a22, a23, a30, a31, a32, a33 = sp.symbols('a00 a01 a02 a03 a10 a11 a12 a13 a20 a21 a22 a23 a30 a31 a32 a33')

x, y = tck_poly[:2]

p = a00 + a01*y + a02*y**2 + a03*y**3 + a10*x + a11*x*y + a12*x*y**2 + a13*x*y**3 + a20*x**2 + a21*x**2*y + a22*x**2*y**2 + a23*x**2*y**3 + a30*x**3 + a31*x**3*y + a32*x**3*y**2 + a33*x**3*y**3

px = sp.diff(p, x)
py = sp.diff(p, y)
pxy = sp.diff(p, x, y)


df = pd.DataFrame(columns=['function', 'evaluation'])

for i in range(2):
    for j in range(2):
        function = 'p({}, {})'.format(j,i)
        df.loc[len(df)] = [function, p.subs({x:j, y:i})]
for i in range(2):
    for j in range(2):
        function = 'px({}, {})'.format(j,i)
        df.loc[len(df)] = [function, px.subs({x:j, y:i})]
for i in range(2):
    for j in range(2):
        function = 'py({}, {})'.format(j,i)
        df.loc[len(df)] = [function, py.subs({x:j, y:i})]
for i in range(2):
    for j in range(2):
        function = 'pxy({}, {})'.format(j,i)
        df.loc[len(df)] = [function, pxy.subs({x:j, y:i})]

eqns = df['evaluation'].tolist()
symbols = [a00, a01, a02, a03, a10, a11, a12, a13, a20, a21, a22, a23, a30, a31, a32, a33]
A = sp.linear_eq_to_matrix(eqns, *symbols)[0]
A = np.array(A.inv()).astype(np.float64)

print(df)

print(A)
print(A.shape)

In [ ]:
# Define a function to calculate the bicubic coefficients
def bicubic_coeffs(pixels):
    # Construct the B matrix
    B = np.array(pixels)

    # Construct the C matrix
    C = np.array([[1, 0, 0, 0],
                  [0, 0, 1, 0],
                  [-3, 3, -2, -1],
                  [2, -2, 1, 1]])

    # Calculate the coefficients
    coeffs = np.dot(np.dot(C, B), C.T)

    # Return the coefficients as a 1D array
    return coeffs.flatten()

# Define a function to calculate the bicubic coefficients for a 2D array of pixels
def bicubic_coeffs_2d(img):
    # Get the dimensions of the image
    rows, cols = img.shape
    lines = (rows-4) * (cols-4)
    # Initialize the coefficient matrix
    A = np.zeros(lines, 16)

    # Loop through each pixel in the image
    k = 0
    for i in range(2, rows-2):
        for j in range(2, cols-2):
            # Extract the 16 pixels surrounding the current pixel
            pixels = img[i-2:i+2+1, j-2:j+2+1].flatten()

            # Calculate the bicubic coefficients for these pixels
            coeffs = bicubic_coeffs(pixels)

            # Add the coefficients to the matrix
            A[k, :] = coeffs
            k += 1

    # Calculate the least squares solution to the coefficient matrix
    b = img[2:rows-2, 2:cols-2].flatten()
    x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

    # Return the coefficients as a 2D array
    return x.reshape(rows-4, cols-4, 16), A

In [ ]:
import cv2

# Load an image
img = cv2.imread('img.png', cv2.IMREAD_GRAYSCALE)

# Calculate the bicubic coefficients
coeffs = bicubic_coeffs_2d(img)
coeffs